# Example Usage

In [1]:
import pandas as pd
import logfile_parser
df = logfile_parser.logfile2dataframe("20150106.log")

In [2]:
df.head()

,severity,server,state,message
time,,,,
2015-01-06 08:04:47.568271,I,datalogger,NaN,Opened: /raid10/aux/2015/01/06/20150106.log
2015-01-06 08:04:47.570016,I,datalogger,NaN,Opened: /raid10/aux/2015/01/06/20150106.rep
2015-01-06 08:04:47.244000,I,DRIVE_CONTROL,NaN,Next sun-rise will be at 2015-Jan-07 08:04:54....
2015-01-06 08:04:47.574848,I,datalogger,NaN,Opened: /raid10/aux/2015/01/06/20150106.DRIVE_...
2015-01-06 08:04:47.581557,I,datalogger,NaN,Opened: /raid10/aux/2015/01/06/20150106.FTM_CO...


In [3]:
df.dtypes

severity    category
server      category
state        float64
message       object
dtype: object

In [4]:
df.server.value_counts()

MAGIC_WEATHER    4537
BIAS_CONTROL     2938
DIM_CONTROL      1515
TEMPERATURE      1440
FSC_CONTROL       699
FEEDBACK          451
FAD_CONTROL       416
MCP               403
FTM_CONTROL       322
datalogger        264
TIME_CHECK        192
RATE_CONTROL      140
DRIVE_CONTROL     120
RATE_SCAN          42
MAGIC_LIDAR        36
LID_CONTROL        32
EVENT_SERVER       27
DATA_LOGGER        24
TNG_WEATHER        12
PWR_CONTROL         2
CHAT                1
dtype: int64

In [5]:
df.severity.value_counts()

-    10004
I     1426
#     1389
W      763
E       31
dtype: int64

# Lets have a look at the Errors only

In [6]:
errors = df[df.severity == "E"]

In [7]:
pd.DataFrame(logfile_parser.round_time(errors).groupby(["rounded_time", "server"]).message.sum())

/home/dneise/fact/logfile_parser/logfile_parser/logfile_parser.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["rounded_time"] = pd.DatetimeIndex((df.index.asi8//round_to_nanoseconds).round() * round_to_nanoseconds)


,,message
rounded_time,server,
2015-01-06 19:03:41,DIM_CONTROL,| scripts/handleDriveArmed.js: l.44: Error: Wa...
2015-01-06 19:25:04,FEEDBACK,Current limit for shutdown exceeded.... swtchi...
2015-01-06 19:26:04,FEEDBACK,Current limit for shutdown exceeded.... swtchi...
2015-01-06 19:26:43,FEEDBACK,Current limit for shutdown exceeded.... swtchi...
2015-01-06 19:26:46,BIAS_CONTROL,CheckChannelVoltage - Set voltage 75.8715V of ...
2015-01-06 19:26:52,BIAS_CONTROL,CheckChannelVoltage - Set voltage 76.1911V of ...
2015-01-06 19:26:55,FEEDBACK,Current limit for shutdown exceeded.... swtchi...
2015-01-06 19:28:09,DIM_CONTROL,| scripts/Main.js: l.904: Error: Something une...
2015-01-06 19:32:14,BIAS_CONTROL,Corrupted answer (id=0): received wrap counter...


# Now the Warnings only

In [8]:
warnings = df[df.severity == "W"]

In [9]:
warnings.server.value_counts()

FSC_CONTROL      699
BIAS_CONTROL      61
FEEDBACK           2
FTM_CONTROL        1
CHAT               0
DATA_LOGGER        0
DIM_CONTROL        0
DRIVE_CONTROL      0
EVENT_SERVER       0
FAD_CONTROL        0
datalogger         0
TNG_WEATHER        0
MAGIC_LIDAR        0
MAGIC_WEATHER      0
MCP                0
PWR_CONTROL        0
RATE_CONTROL       0
RATE_SCAN          0
TEMPERATURE        0
TIME_CHECK         0
LID_CONTROL        0
dtype: int64

In [10]:
fsc_warnings = warnings[warnings.server == "FSC_CONTROL"]
fsc_warnings.head()

,severity,server,state,message
time,,,,
2015-01-06 08:04:49.443,W,FSC_CONTROL,NaN,"Checksum error (V:e34b|ee1b, R:----|----, Nok..."
2015-01-06 08:07:02.416,W,FSC_CONTROL,NaN,"Checksum error (V:a481|e5d9, R:6814|ffff, Nok..."
2015-01-06 08:09:00.614,W,FSC_CONTROL,NaN,"Checksum error (V:05b0|ea05, R:----|----, Nok..."
2015-01-06 08:11:13.588,W,FSC_CONTROL,NaN,"Checksum error (V:894a|e21f, R:20b6|ffff, Nok..."
2015-01-06 08:13:11.845,W,FSC_CONTROL,NaN,"Checksum error (V:7234|e68f, R:f654|ff00, Nok..."


These **Checksum error** we know very well, let's see if there are also other FSC warnings, not about the Checksum

In [11]:
fsc_warnings.message.str.slice(0, 15).value_counts()

Checksum error     680
Suspicious temp     19
Name: message, dtype: int64

Aha some **Suspicious temperatures** were measured also.

In [12]:
bias_warnings = warnings[warnings.server == "BIAS_CONTROL"]
bias_warnings.message.value_counts()

OverCurrent detected - ramping stopped.                                                      48
RampOneStep while waiting for answer to last command (id=1)... ramp step delayed.             4
REQUEST_STATUS: Not allowed in state Ramping[5]... ignored.                                   2
CheckChannelVoltage - Set voltage 77.326V of channel 11 exceeds absolute limit of 76.5V.      1
CheckChannelVoltage - Set voltage 78.1464V of channel 11 exceeds absolute limit of 76.5V.     1
CheckChannelVoltage - Set voltage 81.2434V of channel 10 exceeds absolute limit of 76.5V.     1
CheckChannelVoltage - Set voltage 77.5308V of channel 11 exceeds absolute limit of 76.5V.     1
CheckChannelVoltage - Set voltage 76.5432V of channel 11 exceeds absolute limit of 76.5V.     1
CheckChannelVoltage - Set voltage 77.2282V of channel 51 exceeds absolute limit of 76.5V.     1
CheckChannelVoltage - Set voltage 77.0306V of channel 11 exceeds absolute limit of 76.5V.     1
Name: message, dtype: int64

In [13]:
warnings[warnings.server == "FEEDBACK"]

,severity,server,state,message
time,,,,
2015-01-07 05:38:57.115,W,FEEDBACK,NaN,Critical current limit exceeded timeout.... sw...
2015-01-07 06:14:21.562,W,FEEDBACK,NaN,Critical current limit exceeded timeout.... sw...


In [14]:
warnings[warnings.server == "FTM_CONTROL"]

,severity,server,state,message
time,,,,
2015-01-07 06:44:11.051,W,FTM_CONTROL,NaN,STOP_TRIGGER: Not allowed in state Valid[4]......
